In [ ]:
import pandas as pd
import os
import statsmodels.api as sm
import numpy as np

In [ ]:
#matched samples
bucket=os.getenv('WORKSPACE_BUCKET')

In [ ]:
import pandas
import os

# This query represents dataset "all wgs surveys" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_73282597_survey_sql = """
    SELECT DISTINCT
        answer.person_id,
        answer.question_concept_id,
        answer.question,
        answer.answer,
        answer.survey
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer  
    WHERE
        (
            survey IN (
                "Personal and Family Health History"
            ) 
        )
 """

dataset_73282597_survey_df = pandas.read_gbq(
    dataset_73282597_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_73282597_survey_df.head(5)

In [ ]:
import pandas
import os

# This query represents dataset "survey patients" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_46202400_person_sql = """
    SELECT
        person.person_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN (836793) 
                    AND is_standard = 0  
                    AND  value_source_concept_id IN (1384519))) criteria ) )"""

dataset_46202400_person_df = pandas.read_gbq(
    dataset_46202400_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_46202400_person_df.head(5)

In [ ]:
import pandas
import os

# This query represents dataset "ced2" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_58020306_person_sql = """
    SELECT
        person.person_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        p_race_concept.concept_name as race,
        p_ethnicity_concept.concept_name as ethnicity,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN (836793) 
                    AND is_standard = 0  
                    AND  value_source_concept_id IN (1384519))) criteria 
            UNION
            DISTINCT SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (194992)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )"""

ced_total = pandas.read_gbq(
    dataset_58020306_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

ced_total.head(5)

In [ ]:
#filter for out ehr patients
dataset_73282597_survey_df['label']=0
dataset_73282597_survey_df.loc[dataset_73282597_survey_df['person_id'].isin(dataset_46202400_person_df['person_id']),'label']=1
survey_df=dataset_73282597_survey_df[~dataset_73282597_survey_df['person_id'].isin(ced_total)|(dataset_73282597_survey_df['label']==1)]
survey_df.drop_duplicates('person_id').value_counts('label')

In [ ]:
#filter for those who didn't skip all questions
pfh=survey_df[~survey_df['answer'].str.contains('PMI')]

In [ ]:
len(survey_df.question.value_counts())

In [ ]:
pfh.drop_duplicates('person_id').value_counts('label')

In [ ]:
#other demographics
import pandas
import os

# This query represents dataset "ced2" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_70171923_person_sql = """
    SELECT
        person.person_id,
        person.birth_datetime as date_of_birth,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
  """

demo_person_df = pandas.read_gbq(
    dataset_70171923_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

demo_person_df.head(5)

In [ ]:
#extract enrollment date
dataset_ages_sql = """SELECT DISTINCT

person_id,

observation_datetime AS primary_consent_date

FROM `fc-aou-cdr-prod-ct.C2022Q4R11.concept`

JOIN `fc-aou-cdr-prod-ct.C2022Q4R11.concept_ancestor` on concept_id = ancestor_concept_id

JOIN `fc-aou-cdr-prod-ct.C2022Q4R11.observation` on descendant_concept_id = observation_concept_id

WHERE concept_name = 'Consent PII' AND concept_class_id = 'Module'"""

ages_df = pandas.read_gbq(
    dataset_ages_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")
ages_df.head(5)

In [ ]:
#calculate age
ages_df.sort_values('primary_consent_date',inplace=True)
ages_df.drop_duplicates('person_id',inplace=True)

from datetime import date

def age(birthdate,enrolldate):
    age = enrolldate.year - birthdate.year - ((enrolldate.month, enrolldate.day) < (birthdate.month, birthdate.day))
    return age
from tqdm import tqdm

demo_person_df = pandas.merge(demo_person_df, ages_df, on='person_id', how='left')

demo_person_df['age'] = [age(i,j) for i,j in zip(tqdm(demo_person_df['date_of_birth']),tqdm(demo_person_df['primary_consent_date']))]

In [ ]:
#covariates
covar_df=demo_person_df[demo_person_df['person_id'].isin(pfh.person_id)]

In [ ]:
#annotate # questions answered
pfh_ct = pfh.drop_duplicates(['question','person_id']).value_counts('person_id')

In [ ]:
covars=pd.merge(covar_df,pfh_ct,on='person_id')

In [ ]:
#extract answers indicating specific conditions
survey_reg=pfh[pfh['answer'].str.contains('Self')|pfh['question'].str.contains('Have you ever')]

In [ ]:
survey_reg=survey_reg[~survey_reg['answer'].str.contains('I have/have had no infectious disease|celiac')]

In [ ]:
#counts
counts=survey_reg.value_counts(['answer','label'])

In [ ]:
pfh_counts=pd.DataFrame(counts).reset_index().pivot(columns='label',values='count',index='answer').fillna(0).rename({0:'non-Ced cases',1:'Ced cases'},axis=1)

In [ ]:
#case/control and alternate counts
CeD_total=1789
nonCeD_total=183219
pfh_counts['Ced controls']=1789-pfh_counts['Ced cases']
pfh_counts['non-Ced controls']=183219-pfh_counts['non-Ced cases']


pfh_counts['CeD %']=(pfh_counts['Ced cases']/1789)*100
pfh_counts['non-CeD %']=(pfh_counts['non-Ced cases']/183219)*100

In [ ]:
#calcualte chi-square and Odds ratio
from scipy.stats import chi2_contingency

for i in pfh_counts.index:
    ced_case,ced_ctrl,nonced_case,nonced_ctrl=pfh_counts.loc[i,['Ced cases', 'Ced controls', 'non-Ced cases', 'non-Ced controls']]
    obs=np.array([[ced_case,ced_ctrl],[nonced_case,nonced_ctrl]])
    chi2, p_value, dof, expected = chi2_contingency(obs)
    pfh_counts.loc[i,'Chisq']=chi2
    pfh_counts.loc[i,'p_value univariate']=p_value
    print(chi2, p_value, dof, expected)

pfh_counts['OR univar']=pfh_counts['Ced cases']*pfh_counts['non-Ced controls']/(pfh_counts['non-Ced cases']*pfh_counts['Ced controls'])

In [ ]:
#chisq limited for 3 female specific conditions
print(survey_cat[survey_cat['sex_at_birth']=='Female'].drop_duplicates('person_id').value_counts('label'))
#only female, 116928 cases, 1437 controls

fem = survey_reg[survey_reg['person_id'].isin(covars[covars['sex_at_birth']=='Female'].person_id)]

fem_ct=pd.DataFrame(fem.value_counts(['answer','label'])).reset_index().pivot(columns='label',index='answer',values='count')

from scipy.stats import chi2_contingency

for i in fem_spec:
    ced_case,nonced_case=fem_ct.loc[i,[1,0]]
    ced_ctrl,nonced_ctrl=1437-ced_case,116928-nonced_case
    obs=np.array([[ced_case,ced_ctrl],[nonced_case,nonced_ctrl]])
    chi2, p_value, dof, expected = chi2_contingency(obs)
    pfh_counts.loc[i,'OR univar']=ced_case*nonced_ctrl/(nonced_case*ced_ctrl)
    pfh_counts.loc[i,'Chisq']=chi2
    pfh_counts.loc[i,'p_value univariate']=p_value
    pfh_counts.loc[i,'Ced cases']=ced_case
    pfh_counts.loc[i,'non-Ced cases']=nonced_case
    pfh_counts.loc[i,'Ced controls']=ced_ctrl
    pfh_counts.loc[i,'non-Ced controls']=nonced_ctrl
    pfh_counts.loc[i,'CeD %']=ced_case/1437
    pfh_counts.loc[i,'non-CeD %']=nonced_case/116928
   
    print(i,chi2, p_value, dof, expected,OR)

In [ ]:
#FDR correction
import statsmodels.stats.multitest as st


fdr_df=pfh_counts.reset_index()[['answer','p_value univariate']].drop_duplicates()
#FDR correction as defined previously
def FDR(sample, pval):
    #first sort by pval
    sample.sort_values(by=pval, inplace = True)
    samp = sample.dropna(subset=[pval])
    pvals = samp[pval]
    #calculation of correction
    padj = st.fdrcorrection(pvals, is_sorted=True)
    padded_col = np.pad(padj[1], (0, len(sample) - len(padj[1])), 'constant', constant_values=(np.nan, np.nan))
    sample['Padj'] = padded_col
#execute
FDR(fdr_df,'p_value univariate')

pfh_counts_result = pd.merge(pfh_counts,fdr_df,on=['answer','p_value univariate'],how='left')


In [ ]:
pfh_counts_result.to_csv('personal_family_history_chisq.csv',index=False)

In [ ]:
pfh_counts_result=pd.read_csv('personal_family_history_chisq.csv')

In [ ]:
#updated chi-sq result with conditions to include labelled
survey_phe = pd.read_csv(f'20240622_pfh_chisq_regression.csv')

In [ ]:
#specific conditions selected for regression
survey_reg2=survey_reg[survey_reg['answer'].isin(pfh_counts_result.loc[pfh_counts_result['Padj']<1e-14,'answer'])]

In [ ]:
survey_cat=pd.merge(survey_reg2,covars[['person_id','age','sex_at_birth','count']],on='person_id',how='right')

In [ ]:
survey_cat.to_csv('pfh_survey_regression_input_v2.csv',index=False)

In [ ]:
survey_cat=pd.concat([survey_cat,pd.get_dummies(survey_cat['sex_at_birth']).astype(int)],axis=1)

In [ ]:
#final df for regression
survey_cat.loc[survey_cat['person_id'].isin(dataset_46202400_person_df['person_id']),'label']=1
survey_cat['label'].fillna(0,inplace=True)


In [ ]:
#variables for use in regression
survey_cat['CeD']=survey_cat['label']
covariate=['age', 'Female', 'count']
yvar='CeD'
xvars=['status','age', 'Female', 'count']

In [ ]:
reg_df=pd.DataFrame()

In [ ]:
len(survey_cat['answer'].dropna().unique())
#out of 45, 4 are 'other/unknown', resulting in 41

In [ ]:
#female-specific calculation
fem_spec=[i for i in survey_phe.loc[survey_phe['Sex_specific']=='female','answer']]

In [ ]:
#logistic regression
l=0
for i in survey_cat['answer'].dropna().unique():
    l+=1
    survey_cat['status']=0
    survey_cat.loc[survey_cat['answer']==i,'status']=1
    cases=survey_cat[survey_cat['status']==1].drop_duplicates('person_id')
    controls=survey_cat[~survey_cat['person_id'].isin(cases['person_id'])].drop_duplicates('person_id')
    df = pd.concat([cases,controls])
    if i in fem_spec:
        df=df[df['Female']==1]
        print(i)
    X=df[xvars]
    if i in fem_spec:
        X.drop(['Female'],axis=1,inplace=True)
    X=sm.add_constant(X)
    #remove redundancies when applicable
    y=df[yvar]
    result = sm.Logit(y, X, missing = 'drop').fit(method='bfgs')
    results_as_html = result.summary().tables[0].as_html()
    results_as_html = result.summary().tables[1].as_html()
    res = pd.read_html(results_as_html, header = 0, index_col=0)[0]
    print(result.summary())
    celtot=df.drop_duplicates('person_id').value_counts('CeD')[1]
    ctrltot=df.drop_duplicates('person_id').value_counts('CeD')[0]
    count=df.value_counts(['CeD','status'])
    count=count.reset_index().pivot(columns='CeD',index='status',values='count').fillna(0)
    print(count)
    k='status'
    reg_df.loc[i,'Ced cases'] =  count.loc[1,1]
    reg_df.loc[i,'Ced controls'] = celtot-count.loc[1,1]   
    reg_df.loc[i,'non-Ced cases'] = count.loc[1,0]   
    reg_df.loc[i,'non-Ced controls'] = ctrltot-count.loc[1,0] 
    reg_df.loc[i,'p_value'] =  result.pvalues[k]
    reg_df.loc[i,'beta_ind'] = result.params[k]
    reg_df.loc[i,'conf_int_1']=res.loc[k]['[0.025']
    reg_df.loc[i,'conf_int_2']=res.loc[k]['0.975]']
    reg_df.loc[i,'converged']=result.converged
    #print(k,count[1,k],celtot-count[1,k],count[0,k],ctrltot-count[0,k])
    #for binary vars, recorded values are 'yes' answers to the question 
reg_df

In [ ]:
#adjusted OR
reg_df['OR logistic']=np.exp(reg_df['beta_ind'])

In [ ]:
reg_df.rename({'p_value':'p_value logistic'},axis=1,inplace=True)

In [ ]:
reg_df.to_csv('regression_pfh_v2.csv')

In [ ]:
reg_df.reset_index(inplace=True)
reg_df.rename({'index':'answer'},axis=1,inplace=True)

In [ ]:
pfh_counts_result=pd.merge(survey_phe[['answer', 'answer.1', 'Sex_specific']],pfh_counts_result,on='answer')

In [ ]:
pfh_counts_result=pd.merge(survey_phe[['answer', 'Include in supplementary']],pfh_counts_result,on='answer')

In [ ]:
#merge chi-sq and regression results
full_result=pd.merge(pfh_counts_result[['answer', 'answer.1', 'Sex_specific', 'Include in supplementary',
       'Ced cases', 'Ced controls', 'non-Ced cases',
       'non-Ced controls', 'Chisq', 'p_value univariate', 'OR univar',
       'Padj', 'CeD %', 'non-CeD %']],reg_df[['answer','p_value logistic', 'beta_ind', 'conf_int_1', 'conf_int_2', 'converged','OR logistic']],on='answer',how='left')

In [ ]:
full_result.loc[full_result['OR univar']<1,'outcome']='protective'
full_result.loc[full_result['OR univar']>1,'outcome']='enriched'
full_result.sort_values('Padj',inplace=True)

In [ ]:
full_result=full_result[['answer','Ced cases', 'Ced controls', 'non-Ced cases','non-Ced controls', 'Chisq', 'p_value univariate', 'OR univar', 'Padj',
       'CeD %', 'non-CeD %', 'p_value logistic', 'beta_ind', 'conf_int_1',
       'conf_int_2', 'converged', 'OR logistic']]

In [ ]:
full_result.to_csv('pfh_chisq_regression_v3.csv',index=False)

In [ ]:
!gsutil cp pfh_chisq_regression_v3.csv $WORKSPACE_BUCKET/data/personal_family_health/